In [2]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

Using TensorFlow backend.


In [142]:
model_path = "/deep/group/awni/ecg_models/default/1527864553-601/0.393-0.887-007-0.147-0.953.hdf5"
data_path = "../dev.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 852/852 [00:00<00:00, 1044.95it/s]


In [151]:
data_path = "../train.json"
with open("../train.json", 'r') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = zip(*counts)[1]
smooth = 100
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)

In [152]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [153]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [150]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print "CINC Average {:3f}".format(np.mean(scores[2][:3]))

             precision    recall  f1-score   support

          A      0.778     0.829     0.803        76
          N      0.918     0.942     0.930       514
          O      0.857     0.750     0.800       240
          ~      0.500     0.773     0.607        22

avg / total      0.878     0.873     0.874       852

CINC Average 0.844141
